# Store Sales - Time Series Forecasting

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

## Knowing the Data

In [2]:
path = "../../kaggle dataset/Store Sales/"

train = pd.read_csv(
    path + 'train.csv',
    usecols=['store_nbr', 'family', 'date', 'sales'],
    dtype={'store_nbr': 'category', 'family': 'category', 'sales': 'float32'},
    parse_dates=['date'],
    infer_datetime_format=True
)
train.date = train.date.dt.to_period('D')
train = train.set_index(['store_nbr', 'family', 'date']).sort_index()
test = pd.read_csv(
    path + 'test.csv',
    usecols=['store_nbr', 'family', 'date'],
    dtype={'store_nbr': 'category', 'family': 'category'},
    parse_dates=['date'],
    infer_datetime_format=True
)
test.date = test.date.dt.to_period('D')
test = test.set_index(['store_nbr', 'family', 'date']).sort_index()
train.head()

sales
store_nbr family     date             
1         AUTOMOTIVE 2013-01-01    0.0
                     2013-01-02    2.0
                     2013-01-03    3.0
                     2013-01-04    3.0
                     2013-01-05    5.0

In [3]:
holidays_events = pd.read_csv(path + 'holidays_events.csv', parse_dates=['date'], infer_datetime_format = True)

holidays_events.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [4]:
oil = pd.read_csv(path + 'oil.csv',parse_dates=['date'], infer_datetime_format = True, index_col='date')

oil.head()

,dcoilwtico
date,
2013-01-01,NaN
2013-01-02,93.14
2013-01-03,92.97
2013-01-04,93.12
2013-01-07,93.20


In [5]:
stores = pd.read_csv(path + 'stores.csv')

stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [6]:
transactions = pd.read_csv(path + 'transactions.csv')

transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


## Feature Engineering

In [7]:
#Oil

calendar = pd.DataFrame(index=pd.date_range('2013-01-01','2017-08-31'))

oil['ma_oil'] = oil['dcoilwtico'].rolling(7).mean()

oil.head(10)

,dcoilwtico,ma_oil
date,,
2013-01-01,NaN,NaN
2013-01-02,93.14,NaN
2013-01-03,92.97,NaN
2013-01-04,93.12,NaN
2013-01-07,93.20,NaN
2013-01-08,93.21,NaN
2013-01-09,93.08,NaN
2013-01-10,93.81,93.218571
2013-01-11,93.60,93.284286


In [8]:
calendar = calendar.merge(oil, how='left', left_index=True, right_index=True)
calendar = calendar['ma_oil'].fillna(method='ffill')

calendar.head(10)

2013-01-01          NaN
2013-01-02          NaN
2013-01-03          NaN
2013-01-04          NaN
2013-01-05          NaN
2013-01-06          NaN
2013-01-07          NaN
2013-01-08          NaN
2013-01-09          NaN
2013-01-10    93.218571
Freq: D, Name: ma_oil, dtype: float64

In [9]:
#Set up day of the week column

calendar = pd.DataFrame(calendar)
calendar['dofw'] = calendar.index.dayofweek

calendar

,ma_oil,dofw
2013-01-01,NaN,1
2013-01-02,NaN,2
2013-01-03,NaN,3
2013-01-04,NaN,4
2013-01-05,NaN,5
...,...,...
2017-08-27,47.720000,6
2017-08-28,47.624286,0
2017-08-29,47.320000,1
2017-08-30,47.115714,2


In [10]:
#Holidays Events

# Good Friday correction
holidays_events['date'] = holidays_events['date'].replace({'2013-04-29': pd.to_datetime('2013-03-29')})
holidays_events = holidays_events.set_index('date').sort_index()
holidays_events = holidays_events[holidays_events.locale == 'National'] 
holidays_events = holidays_events.groupby(holidays_events.index).first()

holidays_events.head()

,type,locale,locale_name,description,transferred
date,,,,,
2012-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,False
2012-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
2012-10-12,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
2012-11-02,Holiday,National,Ecuador,Dia de Difuntos,False
2012-11-03,Holiday,National,Ecuador,Independencia de Cuenca,False


In [11]:
#Set up workday column

calendar['wd'] = True
calendar.loc[calendar.dofw > 4, 'wd'] = False

calendar = calendar.merge(holidays_events, how='left', left_index=True, right_index=True)

calendar.loc[calendar.type == 'Bridge', 'wd'] = False
calendar.loc[calendar.type == 'Work Day', 'wd'] = True
calendar.loc[calendar.type == 'Transfer', 'wd'] = False
calendar.loc[(calendar.type == 'Holiday') & (calendar.transferred == False), 'wd'] = False
calendar.loc[(calendar.type == 'Holiday') & (calendar.transferred == True), 'wd'] = True

calendar.head()

,ma_oil,dofw,wd,type,locale,locale_name,description,transferred
2013-01-01,NaN,1,False,Holiday,National,Ecuador,Primer dia del ano,False
2013-01-02,NaN,2,True,NaN,NaN,NaN,NaN,NaN
2013-01-03,NaN,3,True,NaN,NaN,NaN,NaN,NaN
2013-01-04,NaN,4,True,NaN,NaN,NaN,NaN,NaN
2013-01-05,NaN,5,True,Work Day,National,Ecuador,Recupero puente Navidad,False


In [12]:
# Set up training Data Set

# Start and end date for training data
sdate = '2017-01-01'
edate = '2017-08-15'

# Create target matrix
y = train.unstack(['store_nbr', 'family']).loc[sdate:edate]

y.head()

sales                                                \
store_nbr           1                                                 
family     AUTOMOTIVE BABY CARE BEAUTY BEVERAGES BOOKS BREAD/BAKERY   
date                                                                  
2017-01-01        0.0       0.0    0.0       0.0   0.0     0.000000   
2017-01-02        5.0       0.0    0.0    1434.0   0.0   166.819000   
2017-01-03        4.0       0.0    4.0    3081.0   2.0   519.348022   
2017-01-04        1.0       0.0    4.0    3039.0   2.0   543.250977   
2017-01-05        2.0       0.0    3.0    2617.0   0.0   533.479980   

                                                     ...            \
store_nbr                                            ...         9   
family     CELEBRATION CLEANING   DAIRY        DELI  ... MAGAZINES   
date                                                 ...             
2017-01-01         0.0      0.0     0.0    0.000000  ...       0.0   
2017-01-02         0.0    332.0   376.0   44.980000  ...       5.0   
2017-01-03        15.0    952.0  1045.0  209.300003  ...       2.0   
2017-01-04        17.0   1055.0  1029.0  135.944000  ...       3.0   
2017-01-05        40.0    918.0   853.0  137.005997  ...       2.0   

                                                                           \
store_nbr                                                                   
family           MEATS PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS   
date                                                                        
2017-01-01    0.000000           0.0          0.0                     0.0   
2017-01-02  659.570007        1243.0         11.0                    41.0   
2017-01-03  547.364014         876.0          6.0                    15.0   
2017-01-04  395.287994         677.0          6.0                    13.0   
2017-01-05  470.768005         604.0          7.0                    10.0   

                                                                               \
store_nbr                                                                       
family         POULTRY PREPARED FOODS      PRODUCE SCHOOL AND OFFICE SUPPLIES   
date                                                                            
2017-01-01    0.000000       0.000000     0.000000                        0.0   
2017-01-02  843.596008     115.188995  3136.895996                        1.0   
2017-01-03  714.659973     133.039001  3229.558105                        1.0   
2017-01-04  536.830017      75.201004  1491.416992                        7.0   
2017-01-05  414.100006     113.698997  1566.821045                        1.0   

                    
store_nbr           
family     SEAFOOD  
date                
2017-01-01     0.0  
2017-01-02    23.0  
2017-01-03    14.0  
2017-01-04     0.0  
2017-01-05    17.0  

[5 rows x 1782 columns]

In [13]:
# Deterministic Process
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess, Fourier
import warnings
warnings.filterwarnings("ignore")

fourier = CalendarFourier(freq = 'W', order = 4)

dp = DeterministicProcess(index=y.index,
                          constant=False,
                          order=1,
                          seasonal=False,
                          additional_terms=[fourier],
                          drop=True)
dp

In [14]:
X = dp.in_sample()

X.head()

,trend,"sin(1,freq=W-SUN)","cos(1,freq=W-SUN)","sin(2,freq=W-SUN)","cos(2,freq=W-SUN)","sin(3,freq=W-SUN)","cos(3,freq=W-SUN)"
date,,,,,,,
2017-01-01,1.0,-0.781831,0.623490,-0.974928,-0.222521,-0.433884,-0.900969
2017-01-02,2.0,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
2017-01-03,3.0,0.781831,0.623490,0.974928,-0.222521,0.433884,-0.900969
2017-01-04,4.0,0.974928,-0.222521,-0.433884,-0.900969,-0.781831,0.623490
2017-01-05,5.0,0.433884,-0.900969,-0.781831,0.623490,0.974928,-0.222521


In [15]:
# Adding calendar features to training feature matrix X

X['oil']  = calendar.loc[sdate:edate]['ma_oil'].values
X['dofw'] = calendar.loc[sdate:edate]['dofw'].values
X['wd']   = calendar.loc[sdate:edate]['wd'].values
X['type'] = calendar.loc[sdate:edate]['type'].values

X = pd.get_dummies(X, columns=['dofw'], drop_first=True)
X = pd.get_dummies(X, columns=['type'], drop_first=False)

X.head()

,trend,"sin(1,freq=W-SUN)","cos(1,freq=W-SUN)","sin(2,freq=W-SUN)","cos(2,freq=W-SUN)","sin(3,freq=W-SUN)","cos(3,freq=W-SUN)",oil,wd,dofw_1,dofw_2,dofw_3,dofw_4,dofw_5,dofw_6,type_Additional,type_Event,type_Holiday,type_Transfer
date,,,,,,,,,,,,,,,,,,,
2017-01-01,1.0,-0.781831,0.623490,-0.974928,-0.222521,-0.433884,-0.900969,51.801429,True,0,0,0,0,0,1,0,0,1,0
2017-01-02,2.0,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,51.801429,False,0,0,0,0,0,0,0,0,0,1
2017-01-03,3.0,0.781831,0.623490,0.974928,-0.222521,0.433884,-0.900969,51.801429,True,1,0,0,0,0,0,0,0,0,0
2017-01-04,4.0,0.974928,-0.222521,-0.433884,-0.900969,-0.781831,0.623490,51.801429,True,0,1,0,0,0,0,0,0,0,0
2017-01-05,5.0,0.433884,-0.900969,-0.781831,0.623490,0.974928,-0.222521,51.801429,True,0,0,1,0,0,0,0,0,0,0


## Modelling

I am going to be using a hybrid model.

In [16]:
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

feature_model = Ridge(alpha = 0.5, normalize = True)
target_model = RandomForestRegressor(n_estimators = 250, random_state = 0)

### Training and Evaluation

In [17]:
# Create class for two algorithms
class BoostedHybrid:
    def __init__(self, model_1, model_2):
        self.model_1 = model_1
        self.model_2 = model_2
        self.y_columns = None  # store column names from fit method

    def fit(self, X, y):

        self.model_1.fit(X,y)
        y_fit = pd.DataFrame(self.model_1.predict(X),index=X.index, columns=y.columns)
        y_resid = y - y_fit
        self.model_2.fit(X, y_resid)
        self.y_columns = y.columns
    
    def predict(self, X):
        y_pred1 = pd.DataFrame(self.model_1.predict(X),index=X.index, columns=self.y_columns)
        y_pred2 = pd.DataFrame(self.model_2.predict(X),index=X.index, columns=self.y_columns)     
        y_pred = y_pred1 + y_pred2 
    
        return y_pred

In [18]:
model = BoostedHybrid(feature_model, target_model)
model.fit(X, y)
y_pred= model.predict(X)
y_pred = y_pred.clip(0.0)
y_pred

sales                                                          \
store_nbr           1                                                           
family     AUTOMOTIVE BABY CARE    BEAUTY    BEVERAGES     BOOKS BREAD/BAKERY   
date                                                                            
2017-01-01   0.615251       0.0  0.638163   431.696930  0.295115    61.654456   
2017-01-02   3.215934       0.0  0.209937  1297.854431  0.063036   173.115970   
2017-01-03   3.547626       0.0  4.085490  2903.193178  1.691666   483.705136   
2017-01-04   2.313053       0.0  4.134170  2992.803198  1.690947   529.218379   
2017-01-05   1.970248       0.0  3.872949  2638.419102  0.505927   507.341491   
...               ...       ...       ...          ...       ...          ...   
2017-08-11   2.245518       0.0  1.409699  1137.137328  0.000000   162.712871   
2017-08-12   5.313263       0.0  3.146370  1837.283776  0.000000   275.840245   
2017-08-13   1.933009       0.0  1.367503   831.945530  0.000000   129.119107   
2017-08-14   2.276177       0.0  5.320994  2115.735764  0.002781   341.735219   
2017-08-15   3.684967       0.0  3.977013  1991.759146  0.074747   325.619322   

                                                              ...            \
store_nbr                                                     ...         9   
family     CELEBRATION     CLEANING        DAIRY        DELI  ... MAGAZINES   
date                                                          ...             
2017-01-01    0.749441   116.628653   142.131208   22.586494  ...  1.540292   
2017-01-02    0.723871   318.089744   359.227056   46.599416  ...  3.735509   
2017-01-03   15.432680   919.535215   960.350798  180.221045  ...  2.173824   
2017-01-04   16.356237  1015.562464  1022.672357  149.193221  ...  2.798731   
2017-01-05   30.597720   887.153993   866.170034  132.954599  ...  2.252634   
...                ...          ...          ...         ...  ...       ...   
2017-08-11    4.979794   347.693158   364.806963   73.172026  ...  4.693840   
2017-08-12    5.838509   415.033386   576.756635  106.860859  ...  2.880872   
2017-08-13    0.489359   171.766233   266.539600   49.865680  ...  2.963245   
2017-08-14    7.125987   579.568409   676.974485  144.189159  ...  9.261941   
2017-08-15   16.938295   687.613100   606.011582  119.772758  ...  9.005688   

                                                                           \
store_nbr                                                                   
family           MEATS PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS   
date                                                                        
2017-01-01  175.276324    292.391377     3.350619                7.050805   
2017-01-02  519.171956    953.994920     8.651720               29.169269   
2017-01-03  497.774573    813.053139     5.929647               15.327616   
2017-01-04  429.781622    720.449199     6.765177               14.713960   
2017-01-05  510.511566    616.776701     6.626844               10.650565   
...                ...           ...          ...                     ...   
2017-08-11  323.060336    428.486827     9.628707                5.220791   
2017-08-12  301.192348    458.871216     7.020951               10.307123   
2017-08-13  357.311131    536.276452     5.109374               10.207364   
2017-08-14  332.679910    431.221281     3.493705               10.439837   
2017-08-15  393.988777    476.857479     5.114249                5.969596   

                                                                               \
store_nbr                                                                       
family         POULTRY PREPARED FOODS      PRODUCE SCHOOL AND OFFICE SUPPLIES   
date                                                                            
2017-01-01  240.075108      45.584269   825.083046                   1.484463   
2017-01-02  661.920914      97.171854  2436.815976   

In [19]:
from sklearn.metrics import mean_squared_log_error

y_pred  = y_pred.stack(['store_nbr', 'family']).reset_index()
y_target = y.stack(['store_nbr', 'family']).reset_index().copy()

y_target['sales_pred'] = y_pred['sales']
print('MSLE:', y_target.groupby('family').apply(lambda a: np.sqrt(mean_squared_log_error(a['sales'],a['sales_pred']))).sum())
y_target.groupby('family').apply(lambda a: np.sqrt(mean_squared_log_error(a['sales'],a['sales_pred'])))

MSLE: 10.640344633858486


family
AUTOMOTIVE                    0.253377
BABY CARE                     0.114432
BEAUTY                        0.238006
BEVERAGES                     0.548386
BOOKS                         0.097230
BREAD/BAKERY                  0.392285
CELEBRATION                   0.281018
CLEANING                      0.465674
DAIRY                         0.432688
DELI                          0.347607
EGGS                          0.326092
FROZEN FOODS                  0.312900
GROCERY I                     0.570176
GROCERY II                    0.293948
HARDWARE                      0.237500
HOME AND KITCHEN I            0.297097
HOME AND KITCHEN II           0.244124
HOME APPLIANCES               0.165871
HOME CARE                     0.357274
LADIESWEAR                    0.243454
LAWN AND GARDEN               0.270962
LINGERIE                      0.311232
LIQUOR,WINE,BEER              0.568667
MAGAZINES                     0.237249
MEATS                         0.354918
PERSONAL CARE     

## Training Test Data

In [20]:
# Start and end date for test data
stest = '2017-08-16'
etest = '2017-08-31'

X_test = dp.out_of_sample(steps=16)

# Adding calendar features to test feature matrix X_test

X_test['oil']  = calendar.loc[stest:etest]['ma_oil'].values
X_test['dofw'] = calendar.loc[stest:etest]['dofw'].values
X_test['wd']   = calendar.loc[stest:etest]['wd'].values

X_test = pd.get_dummies(X_test, columns=['dofw'], drop_first=True)

# No national level events in this period
X_test[['type_Additional', 'type_Event', 'type_Holiday', 'type_Transfer' ]] = 0       

sales_pred = pd.DataFrame(model.predict(X_test), index = X_test.index,columns = y.columns)  
sales_pred = sales_pred.stack(['store_nbr', 'family'])

sales_pred

sales
           store_nbr family                                 
2017-08-16 1         AUTOMOTIVE                     4.159539
                     BABY CARE                      0.000000
                     BEAUTY                         4.350294
                     BEVERAGES                   2183.587992
                     BOOKS                          0.205347
...                                                      ...
2017-08-31 9         POULTRY                      301.455109
                     PREPARED FOODS               103.871969
                     PRODUCE                     1181.178246
                     SCHOOL AND OFFICE SUPPLIES   109.336597
                     SEAFOOD                       13.097588

[28512 rows x 1 columns]

In [21]:
submission_hybrid = pd.read_csv(path + 'sample_submission.csv', index_col='id')
submission_hybrid.sales = sales_pred.values
submission_hybrid.to_csv('hybrid.csv', index=True)